In [1]:
import os
from os.path import join
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely import Point
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import xarray as xr
from haversine import haversine, haversine_vector

In [2]:
kilns_path = "/home/patel_zeel/compass24/exact_latlon"
population_path = "/home/patel_zeel/compass24/data_files"

### India Shapefile

In [3]:
gdf = gpd.read_file("/home/rishabh.mondal/Brick-Kilns-project/albk/experiments/data_preperation/shapefiles/statewise/DISTRICT_BOUNDARY.shp")
gdf = gdf.to_crs(epsg=4326)

gdf['District'] = gdf['District'].str.replace('>', 'A').replace('<', 'A')
gdf['STATE'] = gdf['STATE'].str.replace('>', 'A').replace('<', 'A')
gdf.STATE.unique()

array(['GUJARAT', 'MADHYA PRADESH', 'UTTAR PRADESH', 'RAJASTHAN',
       'KERALA', 'DISPUTED (MADHYA PRADESH & GUJARAT)', 'UTTARAKHAND',
       'ANDHRA PRADESH', 'ODISHA', 'KARNATAKA', 'CHHATT|SGARH',
       'HIMACHAL PRADESH', 'MANIPUR', 'JHARKHAND', 'DELHI', 'MIZORAM',
       'CHAND|GARH', 'DADRA & NAGAR HAVELI & DAMAN & DIU', 'TRIPURA',
       'SIKKIM', 'MEGHALAYA', 'DISPUTED (MADHYA PRADESH & RAJASTHAN)',
       'PUDUCHERRY', 'LAKSHADWEEP',
       'DISPUTED (WEST BENGAL , BIHAR & JHARKHAND)', 'ANDAMAN & NICOBAR',
       'GOA', 'JAMMU AND KASHM|R', 'LADAKH', 'TELANGANA', 'MAHARASHTRA',
       'WEST BENGAL', 'HARYANA', 'PUNJAB', 'ARUNACHAL PRADESH', 'BIHAR',
       'NAGALAND', 'TAMIL NADU', 'DISPUTED (RAJATHAN & GUJARAT)', 'ASSAM'],
      dtype=object)

In [4]:
def get_population(state, thresholds):
    print(f"Getting population for {state}")
    gdf_state = gdf[gdf['STATE'] == state.upper().replace("_", " ")]
    gdf_state = gdf_state.to_crs("EPSG:4326").unary_union
    min_lon, min_lat, max_lon, max_lat = gdf_state.bounds
    
    if not os.path.exists(join(population_path, f"{state}.csv")):
        df = xr.open_dataset(join(population_path, "landscan-global-2022.nc")).sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon))['Band1'].to_dataframe().reset_index()
        print("Rectangle data", df.shape)
        
        # Filter out the points that are not in state
        # df['inside'] = Parallel(42)([delayed(lambda lon, lat: True if Point(lon, lat).within(gdf_state) else False)(lon, lat) for lon, lat in tqdm(zip(df['lon'], df['lat']))])
        points = [Point(lon, lat) for lon, lat in tqdm(zip(df['lon'], df['lat']))]
        
        def get(i):
            return gdf_state.contains(points[i:i+10000])

        l_of_l = Parallel(42)([delayed(get)(i) for i in tqdm(range(0, len(points), 10000))])
        inside = []
        for l in l_of_l:
            inside.extend(l)
        
        df['inside'] = inside
        df = df[df['inside']].drop(columns=['inside'])
        df.to_csv(join(population_path, f"{state}.csv"), index=False)
    else:
        print("Reading from file for state", state)
        df = pd.read_csv(join(population_path, f"{state}.csv"))
    print("State data", df.shape)
    
    pop_latlons = df[['lat', 'lon']].values
    kilns = pd.read_csv(join(kilns_path, f"{state}.csv"))
    kiln_latlons = kilns[['lat', 'lon']].values
    print("Calculating distances")
    distances = haversine_vector(pop_latlons, kiln_latlons, unit='km', comb=True)
    pop_dict = {'Total': df['Band1'].sum()}
    for pop in thresholds:
        print(f"Calculating for {pop}")
        within = np.any(distances < pop, axis=0)
        df['within'] = within
        pop_dict[pop] = df[df['within']]['Band1'].sum()
    print(pop_dict)
    return pop_dict

states = ["himachal_pradesh", "punjab", "haryana", "bihar", "jharkhand", "madhya_pradesh", "west_bengal", "uttar_pradesh", "uttarakhand"]
thresholds = [0.8, 2, 5]

state_res = {}
for state in states:
    state_res[state] = get_population(state, thresholds)

Getting population for himachal_pradesh
Reading from file for state himachal_pradesh
State data (76719, 3)
Calculating distances
Calculating for 0.8
Calculating for 2
Calculating for 5
{'Total': 7613055.0, 0.8: 175102.0, 2: 680473.0, 5: 2109156.0}
Getting population for punjab
Reading from file for state punjab
State data (68391, 3)
Calculating distances
Calculating for 0.8
Calculating for 2
Calculating for 5
{'Total': 31038997.0, 0.8: 1953060.0, 2: 10032811.0, 5: 25639927.0}
Getting population for haryana
Reading from file for state haryana
State data (59057, 3)
Calculating distances
Calculating for 0.8
Calculating for 2
Calculating for 5
{'Total': 29626311.0, 0.8: 1124244.0, 2: 6338394.0, 5: 19358785.0}
Getting population for bihar
Reading from file for state bihar
State data (121932, 3)
Calculating distances
Calculating for 0.8
Calculating for 2
Calculating for 5
{'Total': 124896461.0, 0.8: 9428655.0, 2: 44216411.0, 5: 98409812.0}
Getting population for jharkhand
Reading from file f

In [5]:
for state in states:
    state_res[state] = {str(key): val for key, val in state_res[state].items()}
pop_df = pd.DataFrame(state_res).T
pop_df.index.name = "state"

# convert to million and billions
pop_df = pop_df.sort_values("Total", ascending=False)
pop_df.loc["Total"] = pop_df.sum()

print(f"Our study involves {pop_df.loc['Total', 'Total'] / 1.41e9 * 100:.2f}% of the Indian population.")

print(f"{pop_df.loc['Total', '0.8'] / pop_df.loc['Total', 'Total'] * 100:.2f}% population lives within 800 m of the brick kilns.")

print(f"{pop_df.loc['Total', '2'] / pop_df.loc['Total', 'Total'] * 100:.2f}% population lives within 2 km of the brick kilns.")

pop_df = pop_df.applymap(lambda x: f"{x / 1e6:.2f} M" if x > 1e6 else f"{x / 1e9:.2f} B" if x > 1e9 else f"{x / 1e3:.2f} K")
pop_df = pop_df[["0.8", "2", "5", "Total"]]
pop_df = pop_df.rename(columns={"0.8": "< 0.8 km", "2": "< 2 km", "5": "< 5 km", "Total": "Total Population"})
# rename states
pop_df = pop_df.rename(index={"punjab": "Punjab", "haryana": "Haryana", "bihar": "Bihar", "jharkhand": "Jharkhand", "madhya_pradesh": "Madhya Pradesh", "west_bengal": "West Bengal", "uttar_pradesh": "Uttar Pradesh", "himachal_pradesh": "Himachal Pradesh", "uttarakhand": "Uttarakhand"})

md = pop_df.to_markdown(numalign="right", stralign="right")
md_splits = md.split("\n")
md_splits[1] = md_splits[1].replace("|-", "|:-", 1).replace(":|", "|", 1)
md = "\n".join(md_splits)
print(md)

Our study involves 47.06% of the Indian population.
4.80% population lives within 800 m of the brick kilns.
22.29% population lives within 2 km of the brick kilns.
|            state |   < 0.8 km |   < 2 km |   < 5 km |   Total Population |
|:-----------------|-----------:|---------:|---------:|-------------------:|
|    Uttar Pradesh |    13.81 M |  63.32 M | 168.83 M |           233.00 M |
|            Bihar |     9.43 M |  44.22 M |  98.41 M |           124.90 M |
|      West Bengal |     4.35 M |  18.54 M |  50.47 M |           102.10 M |
|   Madhya Pradesh |   258.06 K |   1.40 M |   5.84 M |            84.69 M |
|        Jharkhand |   406.06 K |   2.04 M |   8.32 M |            38.94 M |
|           Punjab |     1.95 M |  10.03 M |  25.64 M |            31.04 M |
|          Haryana |     1.12 M |   6.34 M |  19.36 M |            29.63 M |
|      Uttarakhand |   319.41 K |   1.31 M |   3.91 M |            11.64 M |
| Himachal Pradesh |   175.10 K | 680.47 K |   2.11 M |           

|            state |   < 0.8 km |   < 2 km |   < 5 km |   Total Population |
|-----------------:|-----------:|---------:|---------:|-------------------:|
|    Uttar Pradesh |    13.81 M |  63.32 M | 168.83 M |           233.00 M |
|            Bihar |     4.00 M |  19.76 M |  58.08 M |           124.90 M |
|      West Bengal |     4.35 M |  18.54 M |  50.47 M |           102.10 M |
|   Madhya Pradesh |   258.06 K |   1.40 M |   5.84 M |            84.69 M |
|        Jharkhand |   406.06 K |   2.04 M |   8.32 M |            38.94 M |
|           Punjab |     1.95 M |  10.03 M |  25.64 M |            31.04 M |
|          Haryana |     1.12 M |   6.34 M |  19.36 M |            29.63 M |
|      Uttarakhand |   319.41 K |   1.31 M |   3.91 M |            11.64 M |
| Himachal Pradesh |   175.10 K | 680.47 K |   2.11 M |             7.61 M |
|            Total |    26.40 M | 123.42 M | 342.55 M |           663.55 M |